# Exercise 2#

## Task 1 ##


a) Tokenize the sentences in fileyelp_polarity.txt, removing punctuation.

In [30]:
import string
import collections, functools, operator 
import pandas as pd
import numpy as np

def read_file(filename):
    df = pd.read_csv(filename, sep="\t", header=None)
    df = df.assign(tokens =df[0].apply(lambda x : x.translate(str.maketrans('', '', string.punctuation)).lower().split()))
    return df

df = read_file("yelp_polarity.txt")
display(df)

,0,1,tokens
0,Wow... Loved this place.,1,"[wow, loved, this, place]"
1,Crust is not good.,0,"[crust, is, not, good]"
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nasty]"
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho..."
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an..."
...,...,...,...
995,I think food should have flavor and texture an...,0,"[i, think, food, should, have, flavor, and, te..."
996,Appetite instantly gone.,0,"[appetite, instantly, gone]"
997,Overall I was not impressed and would not go b...,0,"[overall, i, was, not, impressed, and, would, ..."
998,"The whole experience was underwhelming, and I ...",0,"[the, whole, experience, was, underwhelming, a..."


The file is read using pandas and saved in the dataframe df. A function is then applied to all of the sentences using the apply() funciton. The lambda-function used first removes punctuation, then applies lower() and split().

b) Calculate token and bigram counts

In [31]:
def tokens_to_token_freq(tokens):
    dictionary = dict.fromkeys(tokens)
    for i in list(dictionary):
        count = 0
        for j in tokens:
            if j==i:
                count += 1
        dictionary[i] = count
    return dictionary

def bigram_list(sent):
    bigrams=[]
    for i in range(0, len(sent) - 1):
        bigrams.append(sent[i] + ' ' + sent[i + 1])
    return bigrams

df = df.assign(bigrams = df["tokens"].apply(lambda x : dict.fromkeys(bigram_list(x))))
df = df.assign(token_count = df["tokens"].apply(lambda x : tokens_to_token_freq(x)))
df = df.assign(bigram_count = df["bigrams"].apply(lambda x : tokens_to_token_freq(x)))
display(df)

,0,1,tokens,bigrams,token_count,bigram_count
0,Wow... Loved this place.,1,"[wow, loved, this, place]","{'this place': None, 'wow loved': None, 'loved...","{'place': 1, 'loved': 1, 'this': 1, 'wow': 1}","{'this place': 1, 'wow loved': 1, 'loved this'..."
1,Crust is not good.,0,"[crust, is, not, good]","{'crust is': None, 'is not': None, 'not good':...","{'crust': 1, 'is': 1, 'good': 1, 'not': 1}","{'crust is': 1, 'is not': 1, 'not good': 1}"
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nasty]","{'tasty and': None, 'just nasty': None, 'not t...","{'nasty': 1, 'texture': 1, 'the': 1, 'tasty': ...","{'tasty and': 1, 'just nasty': 1, 'texture was..."
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho...","{'the late': None, 'and loved': None, 'loved i...","{'recommendation': 1, 'holiday': 1, 'the': 1, ...","{'the late': 1, 'and loved': 1, 'by during': 1..."
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an...","{'on the': None, 'was great': None, 'great and...","{'on': 1, 'were': 1, 'selection': 1, 'menu': 1...","{'on the': 1, 'and so': 1, 'great and': 1, 'me..."
...,...,...,...,...,...,...
995,I think food should have flavor and texture an...,0,"[i, think, food, should, have, flavor, and, te...","{'both were': None, 'should have': None, 'and ...","{'texture': 1, 'lacking': 1, 'flavor': 1, 'i':...","{'have flavor': 1, 'should have': 1, 'and both..."
996,Appetite instantly gone.,0,"[appetite, instantly, gone]","{'instantly gone': None, 'appetite instantly':...","{'appetite': 1, 'gone': 1, 'instantly': 1}","{'instantly gone': 1, 'appetite instantly': 1}"
997,Overall I was not impressed and would not go b...,0,"[overall, i, was, not, impressed, and, would, ...","{'go back': None, 'was not': None, 'not go': N...","{'back': 1, 'i': 1, 'would': 1, 'overall': 1, ...","{'go back': 1, 'was not': 1, 'not impressed': ..."
998,"The whole experience was underwhelming, and I ...",0,"[the, whole, experience, was, underwhelming, a...","{'and i': None, 'experience was': None, 'well ...","{'next': 1, 'the': 1, 'underwhelming': 1, 'exp...","{'and i': 1, 'experience was': 1, 'well just':..."


c) Print the 10 most frequent bigrams and their frequency

In [34]:
def frequency(column):
    frequency = dict(functools.reduce(operator.add, 
         map(collections.Counter, column)))
    frequency = sorted(frequency.items(), key=lambda x: x[1])
    return(list(frequency))

bigrams = frequency(df["bigram_count"])
print(bigrams[len(bigrams) - 10 : ])

[('on the', 23), ('the service', 24), ('service was', 26), ('and i', 27), ('of the', 29), ('food was', 32), ('and the', 38), ('it was', 41), ('the food', 47), ('this place', 72)]


d.) Print a bigram matrix from the 10 most frequent tokens

In [100]:
token_frequency = frequency(df["token_count"])

def bigram_matrix(token_list, bigram_frequencies):
    matrix = np.zeros((len(token_list), len(token_list)))
    for i in range(0, len(matrix)):
        for j in range(0, len(matrix)):
            key_str = token_list[i][0] + " " + token_list[j][0]
            bigram_frequencies = dict(bigram_frequencies)
            if key_str in bigram_frequencies:
                matrix[i, j] = bigram_frequencies[key_str]          
    return matrix

print(bigram_matrix(token_frequency[len(token_frequency) - 10 : ], bigrams))

[[ 0.  1.  4.  0.  0. 10.  0.  0.  0. 29.]
 [ 0.  0.  0. 10.  2.  1. 41.  2.  3.  0.]
 [ 0.  0.  0. 20.  1.  0. 10.  0.  0.  0.]
 [ 1.  0.  0.  0.  1. 16.  1.  0.  0. 10.]
 [ 0.  1.  2.  0.  0.  3.  0.  0.  1. 11.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 21.  0.  0.  0. 13.]
 [ 0.  0.  0.  0.  0.  0. 19.  0.  0.  0.]
 [ 0. 17.  4.  0.  0.  3.  5. 27.  0. 38.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


## Task 2

Using the code of Task 1:

a.) Extract top 20 most frequent bigrams of all positive and negative documents in folder polarity




b.) Calculate their probability as aforementioned

c.) Write a script to save in a file the most probably bigrams of both classes and their probabilites separated by tabs

d.) Compare the output of both classes and write an analysis of your obersvations